In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from pathlib import Path
import pandas as pd
import numpy as np
import itertools
from copy import copy
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import abc
import sys
import re
sys.path.append("../")
import gpusims
import gpusims.plot.metrics as metric
from gpusims.plot.data import PlotData
from gpusims.config import Config, parse_configs
from gpusims.bench import parse_benchmarks

In [21]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [22]:
root_dir = Path("/Users/roman/dev/gpgpusims")
benchmark_dir = root_dir / "benchmarks"
run_dir = root_dir / "run"
assert benchmark_dir.is_dir()
assert run_dir.is_dir()

In [23]:
configs = parse_configs(benchmark_dir / "configs" / "configs.yml")
benchmarks = parse_benchmarks(benchmark_dir / "benchmarks.yml")

pprint(configs)
pprint(benchmarks)
pprint(gpusims.SIMULATORS)

{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transpose),
 'cuda4-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA4-matrixMul),
 'cuda6-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA6-transpose),
 'vectoradd': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/vectorAdd)}
{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBen

In [24]:
selected_simulators = copy(gpusims.SIMULATORS)
selected_configs = copy(configs)
selected_benchmarks = copy(benchmarks)

# for testing
if False:
    testing_simulators = None
    testing_configs = None
    testing_benchmarks = None
    # testing_simulators = list(gpusims.SIMULATORS.keys())
    # testing_simulators = [gpusims.NATIVE]
    # testing_simulators = [gpusims.MULTI2SIM]
    # testing_configs = ["sm6_gtx1080"]
    # testing_benchmarks = ["babelstream"] # "cuda6-transpose"]
    
    if testing_simulators is not None:
        selected_simulators = {k: v for k, v in gpusims.SIMULATORS.items() if k in testing_simulators}
    if testing_configs is not None:
        selected_configs = {k: v for k, v in configs.items() if k in testing_configs}
    if testing_benchmarks is not None:
        selected_benchmarks = {k: v for k, v in benchmarks.items() if k in testing_benchmarks}

pprint(selected_simulators)
pprint(selected_configs)
pprint(selected_benchmarks)

{'accelsim-ptx': <class 'gpusims.accelsim.AccelSimPTXBenchmarkConfig'>,
 'accelsim-sass': <class 'gpusims.accelsim_sass.AccelSimSASSBenchmarkConfig'>,
 'm2s': <class 'gpusims.multi2sim.Multi2SimBenchmarkConfig'>,
 'macsim': <class 'gpusims.macsim.MacSimBenchmarkConfig'>,
 'native': <class 'gpusims.native.NativeBenchmarkConfig'>,
 'tejas': <class 'gpusims.tejas.TejasBenchmarkConfig'>}
{'sm6_gtx1080': Config(key='sm6_gtx1080', name='GTX 1080', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM6_GTX1080'), spec={'sm_count': 20, 'clock_speed': 1607}),
 'sm86_rtx3070': Config(key='sm86_rtx3070', name='RTX 3070', path=PosixPath('/Users/roman/dev/gpgpusims/benchmarks/configs/SM86_RTX3070'), spec={'sm_count': 46, 'clock_speed': 1132})}
{'babelstream': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/BabelStream),
 'cuda10-matrixmul': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-matrixMul),
 'cuda10-transpose': Benchmark(/Users/roman/dev/gpgpusims/benchmarks/CUDA10-transp

In [298]:
colors= ['#0F8C79','#BD2D28','#E3BA22','#E6842A','#137B80','#8E6C8A','#9A3E25', '#E6842A']
colors= ["#5c0048", "#584588", "#3e78b7", "#39a8d1", "#70d5dd", "#b9ffe9"]
sim_color = {
    "GPUTejas": colors[1], # colors[0] not a nice color
    "AccelSim PTX": colors[2],
    "AccelSim SASS": colors[3],
    "Hardware": "#fe5901", # colors[3],
    "Multi2Sim": colors[4],
    "MacSim": colors[5],
}
abbr = {
    "GPUTejas": "Tejas",
    "AccelSim PTX": "AS PTX",
    "AccelSim SASS": "AS SASS",
    "Hardware": "HW",
    "Multi2Sim": "M2S",
    "MacSim": "MS",
}

In [317]:
def plot_bars(data, debug=False, fontsize=14, font_family="Helvetica"):
    metrics = {
        "Cycles": gpusims.plot.metrics.Cycles(data),
        # "Execution Time": gpusims.plot.metrics.ExecutionTime(data),
        #"Total L2 Read Hits": gpusims.plot.metrics.L2ReadHit(data),
        #"Total DRAM Reads": gpusims.plot.metrics.DRAMReads(data),
        #"Total DRAM Writes": gpusims.plot.metrics.DRAMWrites(data),
        #"Total IPC": gpusims.plot.metrics.IPC(data),
        #"Total Instruction Count": gpusims.plot.metrics.InstructionCount(data),
    }
    if False:
        for metric_name, metric in metrics.items():
            print("######", metric_name)
            print(metric.compute())
    # return
    
    for metric_name, metric in metrics.items():
        bars = []
        metric_df = metric.compute()
        # return metric_df
        for sim in metric_df.index:
            # add hardware GPU name
            name = sim
            if sim == "Hardware": #  and metric.hw_df is not None:
                # gpu_name = metric.hw_df.index.get_level_values("Device")[0]
                # name += f" ({gpu_name})"
                name += f" ({data.config.name})"
            
            # print(name)
            # print("x", [f"{data.benchmark.name} {data.inp.args}"])
            # print("y")
            # print(metric_df.loc[sim]["Value"])
            #for stack in metric_df.loc[sim]:
            #     print(stack)
            # return
            bars.append(go.Bar(
                # x= [i],
                # y= [cycles],
                # x should be benchmarks
                x = [f"{data.benchmark.name} {data.inp.args}"], # cycle_df.index,
                y = metric_df.loc[sim],
                text = metric_df.loc[sim],
                textposition='auto',
                # textposition='inside',
                textfont = dict(
                    # color="white",
                    size=fontsize - 2,
                    color="black",
                ),
                # hovertext=name,
                hovertemplate = (
                    "<b>%{x}</b><br>" +
                    "%{y:.2f}<br>"
                ),
                name=str(name),
                # marker=go.bar.Marker(color=colors[i % len(colors)]),
                marker=dict(
                    color=sim_color[sim[0]],
                    # color='rgba(50, 171, 96, 0.6)',
                    line=dict(
                        color='rgba(50, 171, 96, 1.0)',
                        width=0
                    ),
                ),
                # xaxis='x1',
            ))

        margin = 50
        yaxis_text = metric_name
        if metric.config.get("unit") is not None:
            yaxis_text += f" [{metric.config.get('unit')}]"
        layout = go.Layout(
            font_family=font_family,
            font_color="black",
            font_size=fontsize,
            title=dict(
                text=f"{metric_name} for {data.benchmark.name} ({data.config.name})",
                x=0.5,
                y=0.95,
                xanchor="center",
                yanchor="top",
            ),
            yaxis=go.layout.YAxis(
                title=yaxis_text,
                gridcolor="gray",
                zerolinecolor="gray",
                # type="log",
                # range=[min_axis_val * 0.9 ,max_axis_val*1.1]
            ),
            hoverlabel=dict(
                bgcolor="white",
                font_size=fontsize,
                font_family=font_family,
            ),
            barmode="group",
            bargroupgap=0.1,
            bargap=0.25,
            showlegend=True,
            plot_bgcolor="white",
            margin=dict(
                pad=10,
                autoexpand=True,
                l=margin, r=margin, t=1.5*margin, b=margin
            ),
            width=700,
            height=400,
        )
        if metric.config.get("log"):
            layout.yaxis.type = "log"
            # update_yaxes(type="log")
        fig = go.Figure(data=bars, layout=layout)
        # fig.show()
        filename = ["bar", metric_name, data.benchmark.name, data.config.key, data.inp.sanitized_name()]
        filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
        filename = filename.with_suffix(".pdf")
        fig.write_image(filename, format='pdf')

In [318]:
def plot_bars_exec_time(
    data, config,
    debug=False, fontsize=14, font_family="Helvetica",
):
    bars = []
    data = data.sort_values(by=["Simulator", "Benchmark"])
    data = data.set_index(["Simulator", "Kind"])
    print(data)
    
    # simulators = sorted(data.index.get_level_values("Simulator").unique().tolist())
    # simulators = data["Simulator"].unique().tolist()
    simulators = data.index.get_level_values("Simulator").unique().tolist()
    benchmarks = data["Benchmark"]
    
    for i, sim in enumerate(simulators):
        # add hardware GPU name
        name = sim
        if sim == "Hardware":
            name += f" ({config.name})"
        
        # add bars
        # print(data.set_index(["Simulator", "Kind"]))
        # sim_time = data.loc[sim, "Sim"]
        sim_time = data.loc[sim, "Sim"]
        # .set_index(["Simulator", "Kind"])
        # sim_time = sim_time.loc[sim, "Sim"]
        # trace_time = 
        trace_time = data.loc[sim, "Trace"]
        # print(trace_time)
        # print(trace_time.index)
        # print(trace_time.index[0])
        # print(trace_time.index.get_level_values("Simulator"))
        # "Simulator"))
        # print(trace_time.reset_index())
        # sim_time = sim_time.loc[sim, "Sim"]
        # for time_df, color in [(trace_time, "red"), (sim_time, "blue")]:
        bars.append(go.Bar(
            # x = [sim, trace_time[["Benchmark"]]],
            # x = trace_time[["Benchmark"]],
            x = [trace_time[["Benchmark"]], [abbr[sim]] * len(trace_time[["Benchmark"]])],
            # x = i + trace_time[["Benchmark"]] * len(simulators),
            y = trace_time[["Value"]],
            text = trace_time[["Value"]].apply(lambda v: f"tracing<br>{v}"),
            textposition='auto',
            textangle=0,
            showlegend=False,
            # offsetgroup=sim,
            # offset=(i - 1) * 100,
            # width=100,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))
        bars.append(go.Bar(
            # x = [sim, sim_time[["Benchmark"]]],
            # x = [sim_time["Benchmark"], sim],
            # x = sim_time[["Benchmark"]],
            x = [trace_time[["Benchmark"]], [abbr[sim]] * len(trace_time[["Benchmark"]])],
            # x = i + range(len(sim_time[["Benchmark"]])) * len(simulators),
            y = trace_time[["Value"]] + sim_time[["Value"]],
            text = trace_time[["Value"]] + sim_time[["Value"]],
            textposition='auto',
            textangle=0,
            # offsetgroup=sim,
            # offset=(i - 1) * 100,
            # width=100,
            textfont = dict(
                size=fontsize - 2,
                color="black",
            ),
            hovertemplate = (
                "<b>%{x}</b><br>" +
                "%{y:.2f}<br>"
            ),
            name=str(name),
            marker=dict(
                color=sim_color[sim],
                line=dict(
                    color="white",
                    width=2,
                ),
            ),
        ))

    margin = 50
    yaxis_text = metric_name
    if metric.config.get("unit") is not None:
        yaxis_text += f" [{metric.config.get('unit')}]"
    layout = go.Layout(
        font_family=font_family,
        font_color="black",
        font_size=fontsize,
        title=dict(
            text=f"{metric_name} for ({config.name})", # {data.benchmark.name}
            x=0.5,
            y=0.95,
            xanchor="center",
            yanchor="top",
        ),
        yaxis=go.layout.YAxis(
            title=yaxis_text,
            gridcolor="gray",
            zerolinecolor="gray",
            # range=[min_axis_val * 0.9 ,max_axis_val*1.1]
        ),
        xaxis=go.layout.XAxis(
            # title="Benchmark",
            tickfont=dict(
                size=0.8 * fontsize,
            ),
            dividerwidth=0,
            dividercolor="white",
            # tickvals = simulators,
            # ticktext = benchmarks,
            # showticklabels=False,
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=fontsize,
            font_family=font_family,
        ),
        barmode="stack",
        # bargroupgap=0.1,
        bargap=0.1,
        showlegend=True,
        plot_bgcolor="white",
        margin=dict(
            pad=10,
            autoexpand=True,
            l=margin, r=margin, t=1.5*margin, b=margin
        ),
        width=700,
        height=400,
    )
    if metric.config.get("log"):
        layout.yaxis.type = "log"
    return go.Figure(data=bars, layout=layout)

In [319]:
# => per config, benchmark and input, plot bars for each simulator

for (config_name, config), (bench_name, bench) in itertools.product(
    selected_configs.items(),
    selected_benchmarks.items()
):
    print(config_name, bench_name)
    for inp in bench.inputs:
        plot_data = PlotData(benchmark=bench, config=config, inp=inp)
        for (sim_name, sim) in selected_simulators.items():
            if not bench.enabled(sim_name):
                continue
            # print(sim_name, config_name, bench_name)
            bench_config = sim(
                run_dir=run_dir / sim_name.lower(),
                benchmark=bench,
                config=config,
            )
            if not bench_config.input_path(inp).is_dir():
                print(f"WARN: {bench_config.input_path(inp)} does not exist")
                continue
            
            plot_data[sim_name] = bench_config.load_dataframe(inp)
        
        metrics = {
            # "Cycles": gpusims.plot.metrics.Cycles(plot_data),
            "Execution Time": gpusims.plot.metrics.ExecutionTime(plot_data),
            #"Total L2 Read Hits": gpusims.plot.metrics.L2ReadHit(data),
            #"Total DRAM Reads": gpusims.plot.metrics.DRAMReads(data),
            #"Total DRAM Writes": gpusims.plot.metrics.DRAMWrites(data),
            #"Total IPC": gpusims.plot.metrics.IPC(data),
            #"Total Instruction Count": gpusims.plot.metrics.InstructionCount(data),
        }
        for metric_name, metric in metrics.items():
            print("######", metric_name)
            metric_df = metric.compute()
            metric_df["Benchmark"] = benchmark.name
            # break
            # metric_df = metric_df.T # .reset_index()
            # break
            # print(metric_df.index)
            # metric_df = metric_df.rename(column={"Value": bench.name}
            # print(metric_df)
            # for sim_col in metric_df.columns:
            #    for idx_level in metric_df.index:
            #       print(sim_col, idx_level)
            #        print("")
            #        # print(metric_df[metric_df.index.get_level_values(idx_level),sim_col])
            #        print(metric_df.loc[idx_level,sim_col])
            fig = plot_bars_exec_time(metric_df, config=config)
            # fig.show()
            filename = ["bar", metric_name, benchmark.name, config.key, inp.sanitized_name()]
            filename = Path("./figs") / gpusims.utils.slugify("_".join(filename))
            filename = filename.with_suffix(".pdf")
            fig.write_image(filename, format='pdf')
            print("wrote", filename)
        break
    break

# plot_data[gpusims.MULTI2SIM]
# debug.T
metric_df

sm6_gtx1080 vectoradd
###### Execution Time
                        Value        Benchmark
Simulator     Kind                            
AccelSim PTX  Sim    1.000000  CUDA6-transpose
              Trace  0.000000  CUDA6-transpose
AccelSim SASS Sim    1.000000  CUDA6-transpose
              Trace  0.481207  CUDA6-transpose
GPUTejas      Sim    1.883000  CUDA6-transpose
              Trace  0.213806  CUDA6-transpose
Hardware      Sim    0.000004  CUDA6-transpose
              Trace  0.000000  CUDA6-transpose
MacSim        Sim    1.000000  CUDA6-transpose
              Trace  0.000000  CUDA6-transpose
wrote figs/bar_execution-time_cuda6-transpose_sm6_gtx1080_input-1000.pdf


,Simulator,Kind,Value,Benchmark
0,MacSim,Sim,1.000000,CUDA6-transpose
1,MacSim,Trace,0.000000,CUDA6-transpose
2,GPUTejas,Sim,1.883000,CUDA6-transpose
3,GPUTejas,Trace,0.213806,CUDA6-transpose
4,AccelSim PTX,Sim,1.000000,CUDA6-transpose
5,AccelSim PTX,Trace,0.000000,CUDA6-transpose
6,AccelSim SASS,Sim,1.000000,CUDA6-transpose
7,AccelSim SASS,Trace,0.481207,CUDA6-transpose
8,Hardware,Sim,0.000004,CUDA6-transpose
9,Hardware,Trace,0.000000,CUDA6-transpose


In [118]:
# debug dataframe
# sim = gpusims.MULTI2SIM
# sim = gpusims.MACSIM
sim = gpusims.TEJAS
# sim = gpusims.ACCELSIM_SASS
# sim = gpusims.NATIVE
# benchmark = benchmarks["cuda4-matrixmul"]
benchmark = benchmarks["cuda6-transpose"]
# config=configs["sm86_rtx3070"]
config=configs["sm6_gtx1080"]
bench_config = gpusims.SIMULATORS[sim](
    run_dir=run_dir / sim.lower(),
    benchmark=benchmark,
    config=config,
)
assert bench_config.benchmark.enabled(sim)
bench_input = bench_config.benchmark.inputs[0]
df = bench_config.load_dataframe(bench_input)
# pprint(df.columns.tolist())
# df.T[~df.T.index.str.contains(r"SM \d+")]
df # .T

,emulator_type,created,sim_time_secs,total_inst_count,total_cycle_count,dram_total_reads,dram_total_writes,dram_avg_read_latency,total_instr_cache_access,total_const_cache_access,total_shared_cache_access,total_instr_cache_misses,total_const_cache_misses,total_shared_cache_misses,kips,total_ipc,trace_wall_time,sim_wall_time
0,OCELOT,NaN,18.341,2304,9440,0,0,0.0,0,0,0,8,68,0,0.12562,0.244068,0.454692,21.817216


In [37]:
all_bench_configs = []

for (sim_name, sim), (config_name, config), (bench_name, bench) in itertools.product(
    selected_simulators.items(),
    selected_configs.items(),
    selected_benchmarks.items()
):
    if not bench.enabled(sim):
        continue
    
    # for inp in bench:
    #all_plot_configs.append(BenchmarkPlot(bench_config=sim(
    #    run_dir=run_dir / sim_name.lower(),
    #    benchmark=bench,
    #    config=config,
    #)))

print(f"{len(all_bench_configs)} total benchmark configs")

0 total benchmark configs


In [14]:
for plot_config in all_bench_configs:
    for inp in plot_config.bench_config.benchmark.inputs:
        hw_df = plot.load_hardware_df(inp)
        hw_df = plot.load_hardware_df(inp)
        accel_sass_df = pd.read_csv(accel_sass_results / "results/stats.csv")
print("accel sass shape", accel_sass_df.shape)
accel_sass_df = accel_sass_df.pivot(index=["kernel", "kernel_id"], columns=["stat"])["value"]
print("accel sass shape", accel_sass_df.shape)
# pprint(accel_df.columns.tolist())
accel_sass_df.T
        break
    break
hw_df.T
# plot = all_bench_configs[0]

kernels shape (1, 11)
cycles shape (1, 592)
inner join shape (1, 603)


Stream,7.0
Context,1.0
Device,NVIDIA GeForce GTX 1080 (0)
Kernel,_Z6vecAddPdS_S_i
Correlation_ID,112.0
Block X,1024.0
Block Y,1.0
Block Z,1.0
Duration,3.626667
Dynamic SMem,0.0
...,...


In [11]:
plot = all_bench_configs[0]
hw_df = plot.load_hardware_df()

NameError: name 'pd' is not defined

In [ ]:
hw_df = build_hw_df(
    cycle_csv_files=list((native_results / "results").rglob(r"result.cycles.csv.*")),
    kernel_csv_files=list((native_results / "results").rglob(r"result.csv.*")),
)
hw_df.T